## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`. 
Задание выполнить несколькими способами ("naive" Python, numpy, с использованием numba декоратора @njit) Проверить время выполнения каждого способа %time или %%time

In [73]:
from random import randint, choice
import numpy as np
from numba import njit, vectorize

In [311]:
N = 10**6

A = np.array([randint(0, 1000) for _ in range(N)])
B = A + 100

In [205]:
A, B

(array([900, 135, 301, ..., 209, 161, 584]),
 array([1000,  235,  401, ...,  309,  261,  684]))

In [224]:
%time sum(B)/len(B)

CPU times: total: 125 ms
Wall time: 133 ms


599.867313

In [225]:
%time B.mean()

CPU times: total: 15.6 ms
Wall time: 2 ms


599.867313

In [226]:
@njit
def naive(m):
    return sum(m)/len(m)

In [227]:
%time naive(B)

CPU times: total: 141 ms
Wall time: 149 ms


599.867313

In [229]:
%timeit naive(B)

352 µs ± 46.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв. Задание выполнить несколькими способами ("naive" Python, numpy, с использованием numba декоратора @njit - оценить возможность). Проверить время выполнения каждого способа %time или %%time

In [75]:
t = pd.DataFrame([[randint(1, 1000) for _ in range(4)] for _ in range(2_000_000)])

In [76]:
t['key'] = [choice('qwertyuiopasdfghjklzxcvbnm') for _ in range(2_000_000)]

In [77]:
t

,0,1,2,3,key
0,386,744,612,773,v
1,622,118,10,674,w
2,703,401,537,352,i
3,548,330,508,430,v
4,452,406,428,85,t
...,...,...,...,...,...
1999995,873,944,275,258,k
1999996,529,172,996,751,q
1999997,491,230,465,443,k
1999998,848,533,549,982,m


In [299]:
a = pd.DataFrame(columns=[0, 1, 2, 3, 'key'])
for i in t.index:
#     print(i)
    print(t.iloc[i]['key'] in 'abcde')
    a.loc[i] = t.iloc[i]
    break
a

False


,0,1,2,3,key
0,386,744,612,773,v


In [300]:
def naive2(d):
    res = pd.DataFrame(columns=[0, 1, 2, 3, 'key'])
    for i in t.index:
        if t.iloc[i]['key'] in 'abcde':
            res.loc[i] = t.iloc[i]
    return res

In [ ]:
%time naive2(t)

In [267]:
%time t[(t['key'] == 'a')|(t['key'] == 'b')|(t['key'] == 'c')|(t['key'] == 'd')|(t['key'] == 'e')]

CPU times: total: 1 s
Wall time: 1.01 s


,0,1,2,3,key
10,893,162,686,373,b
16,675,592,956,224,b
19,428,145,610,505,c
24,362,9,449,510,b
25,831,556,531,644,b
...,...,...,...,...,...
1999968,888,263,262,153,c
1999969,287,230,381,840,e
1999980,253,80,755,480,c
1999990,176,570,119,211,e


In [91]:
@njit
def naive3(d):
    res = pd.DataFrame(columns=[0, 1, 2, 3, 'key'])
    for i in t.index:
        if t.iloc[i]['key'] in 'abcde':
            res.loc[i] = t.iloc[i]
    return res

In [ ]:
%time naive3(t.to_numpy())

## Лабораторная работа 3

In [3]:
!pip install line_profiler

^C


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [4]:
import pandas as pd

In [46]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

In [47]:
reviews['date'] = reviews['date'].astype("datetime64[ns]")

In [ ]:
# A

In [9]:
def get_mean(d):
    rats = 0
    c = 0
    for i in d.iterrows():
        if '2010' in str(i[1][2]):
            rats += i[1][3]
            c += 1
    return rats/c

In [98]:
%time get_mean(reviews)

CPU times: total: 8.61 s
Wall time: 8.64 s


4.4544402182900615

In [7]:
# Б

reviews_2010 = reviews[('01.01.2010' <= reviews['date']) & (reviews['date'] < '01.01.2011')]

In [8]:
def get_mean_2010(d):
    rats = 0
    for i in d.iterrows():
        rats += i[1][3]
    return rats/len(d)

In [87]:
%time get_mean_2010(reviews_2010)

CPU times: total: 672 ms
Wall time: 683 ms


4.4544402182900615

In [92]:
# B

%time reviews[('01.01.2010' <= reviews['date']) & (reviews['date'] < '01.01.2011')]['rating'].mean()

CPU times: total: 15.6 ms
Wall time: 7 ms


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

Медленне всего выполняется `get_mean()`, т.е. самый первый вариант функции. На скорость выпонения влияет количество вызовов функции

In [12]:
%prun get_mean(reviews)

 17768212 function calls (17261424 primitive calls) in 16.366 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   
   126696    1.289    0.000   11.799    0.000 series.py:342(__init__)
   
        1    1.036    1.036   16.350   16.350 131955222.py:1(get_mean)
        
   126696    0.806    0.000    3.232    0.000 construction.py:493(sanitize_array)
   
  4241482    0.774    0.000    1.224    0.000 {built-in method builtins.isinstance}
  
        1    0.690    0.690    0.690    0.690 {pandas._libs.tslibs.vectorized.ints_to_pydatetime}
        
   138790    0.530    0.000    1.395    0.000 series.py:966(__getitem__)
   
   126696    0.530    0.000    0.780    0.000 generic.py:5844(__finalize__)
   
   126696    0.498    0.000    0.821    0.000 cast.py:1178(maybe_infer_to_datetimelike)
   
2027170/1520382    0.466    0.000    0.735    0.000 {built-in method builtins.len}

   126696    0.454    0.000    1.614    0.000 blocks.py:2172(new_block)
   
   126697    0.431    0.000   13.919    0.000 frame.py:1366(iterrows)
   
   126696    0.424    0.000    0.544    0.000 generic.py:259(__init__)
   
   126697    0.350    0.000    1.460    0.000 generic.py:5904(__setattr__)
   
   126696    0.310    0.000    0.796    0.000 config.py:116(_get_single_key)
   
   126696    0.290    0.000    1.144    0.000 construction.py:744(_try_cast)
   
   138790    0.285    0.000    0.327    0.000 managers.py:2069(internal_values)
   
   126696    0.280    0.000    2.288    0.000 managers.py:1934(from_array)
   
   126696    0.273    0.000    0.463    0.000 blocks.py:2091(maybe_coerce_values)
   
   506788    0.269    0.000    0.330    0.000 base.py:925(__len__)
   
   126696    0.265    0.000    0.338    0.000 blocks.py:2120(get_block_type)
   
   126696    0.243    0.000    0.454    0.000 series.py:611(name)
   
   253392    0.236    0.000    0.236    0.000 config.py:611(_get_deprecated_option)
   
   126696    0.220    0.000    0.655    0.000 series.py:661(name)
   
   126696    0.220    0.000    0.220    0.000 {pandas._libs.lib.infer_datetimelike_array}
   
   126696    0.212    0.000    0.212    0.000 generic.py:5888(__getattr__)

In [10]:
%prun get_mean_2010(reviews_2010)

1681341 function calls (1632959 primitive calls) in 1.779 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   
    12094    0.148    0.000    1.412    0.000 series.py:342(__init__)
    
   399147    0.101    0.000    0.158    0.000 {built-in method builtins.isinstance}
   
    12094    0.094    0.000    0.389    0.000 construction.py:493(sanitize_array)
    12094    0.063    0.000    0.096    0.000 generic.py:5844(__finalize__)
    12094    0.063    0.000    0.169    0.000 series.py:966(__getitem__)
    12094    0.060    0.000    0.096    0.000 cast.py:1178(maybe_infer_to_datetimelike)
    
193544/145162    0.056    0.000    0.090    0.000 {built-in method builtins.len}

    12094    0.053    0.000    0.066    0.000 generic.py:259(__init__)    
    12094    0.052    0.000    0.188    0.000 blocks.py:2172(new_block)    
    12095    0.049    0.000    1.575    0.000 frame.py:1366(iterrows)    
    12095    0.042    0.000    0.175    0.000 generic.py:5904(__setattr__)    
    12094    0.037    0.000    0.096    0.000 config.py:116(_get_single_key)
    12094    0.035    0.000    0.136    0.000 construction.py:744(_try_cast)
    48381    0.034    0.000    0.042    0.000 base.py:925(__len__)
        1    0.034    0.034    1.777    1.777 772986126.py:1(get_mean_2010)
    12094    0.033    0.000    0.039    0.000 managers.py:2069(internal_values)
    12094    0.033    0.000    0.268    0.000 managers.py:1934(from_array)
    12094    0.032    0.000    0.041    0.000 blocks.py:2120(get_block_type)
    12094    0.029    0.000    0.053    0.000 blocks.py:2091(maybe_coerce_values)
    12094    0.029    0.000    0.055    0.000 series.py:611(name)
    24188    0.028    0.000    0.028    0.000 config.py:611(_get_deprecated_option)
    12094    0.026    0.000    0.079    0.000 series.py:661(name)
    12094    0.026    0.000    0.101    0.000 series.py:565(_set_axis)
    36288    0.026    0.000    0.057    0.000 generic.py:45(_instancecheck)

In [11]:
%prun reviews[('01.01.2010' <= reviews['date']) & (reviews['date'] < '01.01.2011')]['rating'].mean()

1512 function calls (1475 primitive calls) in 0.517 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   
        1    0.336    0.336    0.336    0.336 {pandas._libs.algos.take_2d_axis0_object_object}
        1    0.019    0.019    0.019    0.019 {pandas._libs.algos.take_2d_axis1_int64_int64}
        5    0.013    0.003    0.014    0.003 common.py:162(is_object_dtype)
      2/1    0.012    0.006    0.063    0.063 {built-in method _operator.ge}
        4    0.009    0.002    0.401    0.100 frame.py:3758(__getitem__)
        2    0.008    0.004    0.008    0.004 datetimes.py:455(_scalar_from_string)
        2    0.007    0.004    0.064    0.032 datetimelike.py:1048(_cmp_method)
        1    0.006    0.006    0.006    0.006 {method 'take' of 'numpy.ndarray' objects}
        9    0.006    0.001    0.006    0.001 {method 'reduce' of 'numpy.ufunc' objects}
        2    0.005    0.003    0.095    0.047 series.py:6233(_cmp_method)
        3    0.004    0.001    0.004    0.001 datetimes.py:449(_unbox_scalar)
        3    0.004    0.001    0.004    0.001 base.py:5254(__contains__)
        1    0.003    0.003    0.517    0.517 {built-in method builtins.exec}
        4    0.003    0.001    0.003    0.001 datetimes.py:681(_assert_tzawareness_compat)
        6    0.003    0.000    0.003    0.000 {method 'view' of 'numpy.ndarray' objects}
        3    0.003    0.001    0.018    0.006 series.py:3194(_construct_result)
        3    0.002    0.001    0.003    0.001 blocks.py:2172(new_block)
        
  291/281    0.002    0.000    0.004    0.000 {built-in method builtins.isinstance}
  
        1    0.002    0.002    0.514    0.514 <string>:1(<module>)
        2    0.002    0.001    0.003    0.002 common.py:427(is_period_dtype)
        4    0.002    0.000    0.002    0.000 indexing.py:2656(check_deprecated_indexers)
      4/3    0.002    0.000    0.364    0.121 take.py:57(take_nd)
        1    0.002    0.002    0.002    0.002 utils.py:243(maybe_convert_indices)
        5    0.002    0.000    0.002    0.000 generic.py:5844(__finalize__)
        1    0.002    0.002    0.005    0.005 nanops.py:656(nanmean)
        3    0.002    0.001    0.002    0.001 construction.py:744(_try_cast)
      101    0.002    0.000    0.002    0.000 {built-in method builtins.getattr}
        2    0.002    0.001    0.002    0.001 datetimelike.py:882(_isnan)
      2/1    0.002    0.001    0.094    0.094 arraylike.py:60(__ge__)
        4    0.001    0.000    0.014    0.003 series.py:342(__init__)
        8    0.001    0.000    0.003    0.000 _ufunc_config.py:33(seterr)
        1    0.001    0.001    0.008    0.008 base.py:1168(take)

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [108]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [109]:
%time get_word_reviews_count(reviews)

CPU times: total: 18.9 s
Wall time: 19.1 s


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [111]:
%prun get_word_reviews_count(reviews)

52985965 function calls (51972451 primitive calls) in 37.351 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   
        1    7.740    7.740   37.243   37.243 2826575548.py:1(get_word_reviews_count)
        
   253364    2.172    0.000   20.351    0.000 series.py:342(__init__)
   
8995345/8995339    1.429    0.000    2.226    0.000 {built-in method builtins.isinstance}

10846018/9832522    1.346    0.000    1.715    0.000 {built-in method builtins.len}

   253364    1.297    0.000    5.540    0.000 construction.py:493(sanitize_array)
   
   380037    1.045    0.000    4.592    0.000 series.py:966(__getitem__)
   
   253372    0.926    0.000    1.359    0.000 generic.py:5844(__finalize__)
   
   253358    0.866    0.000    1.415    0.000 cast.py:1178(maybe_infer_to_datetimelike)
   
   253376    0.853    0.000    2.790    0.000 generic.py:5904(__setattr__)
   
   506720    0.841    0.000    0.841    0.000 {method 'split' of 'str' objects}
   
   253370    0.752    0.000    2.683    0.000 blocks.py:2172(new_block)
   
  6792020    0.751    0.000    0.751    0.000 {method 'append' of 'list' objects}
  
   253376    0.735    0.000    0.922    0.000 generic.py:259(__init__)
   
   253360    0.714    0.000   22.714    0.000 frame.py:1366(iterrows)
   
   253362    0.536    0.000    1.381    0.000 config.py:116(_get_single_key)
   
   380037    0.521    0.000    2.502    0.000 series.py:1072(_get_value)
   
   253364    0.505    0.000    1.977    0.000 construction.py:744(_try_cast)
   
   380047    0.502    0.000    0.575    0.000 managers.py:2069(internal_values)
   
   253360    0.482    0.000    3.778    0.000 managers.py:1934(from_array)

Во-первых, стоит заметить, что вызываемая функция `iterrows` занимает достаточно много времени, поэтому было бы целесообразно ее заменить. Более того, в функции присутствуют два цикла, из-за чего приходится дважды обходить весь `DataFrame`, это очевидно излишне. Лучше было бы сразу в первом цикле считать количество рецептов, в описании которых есть требуемое слово, и сразу заносить в словарь именно к-во таких рецептов, а не их id

In [176]:
from collections import Counter

In [194]:
def get_word_reviews_count_2(df):
    r = ' '.join(df['review'].dropna())
    return Counter(r.split())

In [195]:
%time get_word_reviews_count_2(reviews)

CPU times: total: 2.58 s
Wall time: 2.59 s


Counter({'Last': 100,
         'week': 804,
         'whole': 5630,
         'sides': 313,
         'of': 109040,
         'frozen': 2648,
         'salmon': 729,
         'fillet': 60,
         'was': 88793,
         'on': 34590,
         'sale': 149,
         'in': 61551,
         'my': 44166,
         'local': 561,
         'supermarket,': 10,
         'so': 46106,
         'I': 288141,
         'bought': 1369,
         'tons': 161,
         '(okay,': 5,
         'only': 13967,
         '3,': 48,
         'but': 42528,
         'total': 381,
         'weight': 160,
         'over': 9066,
         '10': 2305,
         'pounds).': 3,
         'This': 39937,
         'recipe': 41128,
         'is': 55083,
         'perfect': 4400,
         'for': 121248,
         'fillet,': 14,
         'even': 7881,
         'though': 2315,
         'it': 111224,
         'calls': 520,
         'steaks.': 96,
         'cut': 6689,
         'up': 13585,
         'the': 266099,
         'into': 7035,
  

In [179]:
# def get_word_reviews_count_2(df):
#     word_reviews_count = {}
# #     r = df['review']
#     for i in df['review']:
#         if type(i) == str:
# #             words = i.split(' ')
#             for w in i.split(' '):
#                 word_reviews_count[w] = word_reviews_count.setdefault(w, 0) + 1
#     return word_reviews_count

In [180]:
%time get_word_reviews_count_2(reviews)

CPU times: total: 2.72 s
Wall time: 2.83 s


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [107]:
import numba

In [48]:
reviews = reviews[reviews['rating'] != 0]

In [228]:
def MAPE_A(m):
    n = len(m)
    
    exp = 0
    for i in m:
        exp += i
    exp /= n
    
    sigma = 0
    for i in m:
        sigma += abs(i - exp)/i
    
    res = (100/n)*sigma
    
    return res

In [230]:
%time MAPE_A(reviews['rating'])

CPU times: total: 62.5 ms
Wall time: 67 ms


16.266663338761624

In [218]:
@njit
def MAPE_B(m):
    n = len(m)
    
    exp = 0
    for i in m:
        exp += i
    exp /= n
    
    sigma = 0
    for i in m:
        sigma += abs(i - exp)/i
    
    res = (100/n)*sigma
    
    return res

In [221]:
%timeit MAPE_B(reviews['rating'].to_numpy())

313 µs ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [252]:
def MAPE_C(m):
    exp = m.sum()/len(m)
    
    sigma = abs((m - exp)/m).sum()
    
    res = (100/len(m))*sigma
    
    return res

In [264]:
%timeit MAPE_C(reviews['rating'])

1.22 ms ± 58.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [257]:
@njit
def MAPE_D(m):
    exp = m.sum()/len(m)
    
    sigma = (abs(m - exp)/m).sum()
    
    res = (100/len(m))*sigma
    
    return res

In [262]:
%timeit MAPE_D(reviews['rating'].to_numpy())

292 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### [версия 2]
* Уточнены формулировки задач 1, 3, 4